# Калякин Тимофей
# ДЗ 2

## Создание таблиц

In [1]:
import psycopg2 as pg
import pandas as pd
import numpy as np
import pandas as pd

In [3]:

conn = pg.connect(dbname='postgres', host='localhost', user='postgres', password='12345', port=5432)
conn.autocommit = True  
cursor = conn.cursor()

new_db_name = 'hw2_database'
cursor.execute(f'CREATE DATABASE {new_db_name};')

cursor.close()
conn.close()


In [4]:
customer_df = pd.read_csv('customer.csv', delimiter=';')
# Создание новой строки с customer_id = 5034 и остальными значениями как NaN
new_row = {col: None for col in customer_df.columns}
new_row['customer_id'] = int(5034)  
customer_df = pd.concat([customer_df, pd.DataFrame([new_row])], ignore_index=True)
display(customer_df)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,3,Arlin,Dearle,Male,1954-01-20,Recruiting Manager,Property,Mass Customer,N,Yes,0 Holy Cross Court,4211,QLD,Australia,9
3,4,Talbot,NaN,Male,1961-10-03,NaN,IT,Mass Customer,N,No,17979 Del Mar Point,2448,New South Wales,Australia,4
4,5,Sheila-kathryn,Calton,Female,1977-05-13,Senior Editor,NaN,Affluent Customer,N,Yes,9 Oakridge Court,3216,VIC,Australia,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3996,3997,Blanch,Nisuis,Female,2001-07-13,Statistician II,Manufacturing,High Net Worth,N,Yes,87 Crescent Oaks Alley,2756,NSW,Australia,10
3997,3998,Sarene,Woolley,U,NaN,Assistant Manager,IT,High Net Worth,N,No,8194 Lien Street,4032,QLD,Australia,7
3998,3999,Patrizius,NaN,Male,1973-10-24,NaN,Manufacturing,Affluent Customer,N,Yes,320 Acker Drive,2251,NSW,Australia,7
3999,4000,Kippy,Oldland,Male,1991-11-05,Software Engineer IV,NaN,Affluent Customer,N,No,613 Erie Lane,3996,NSW,Australia,10


In [5]:
transaction_df = pd.read_csv('transaction.csv', delimiter=';')
transaction_df['list_price'] = transaction_df['list_price'].astype(str).str.replace(',', '.').astype(float)
transaction_df['standard_cost'] = transaction_df['standard_cost'].astype(str).str.replace(',', '.').astype(float)
transaction_df['customer_id'] = transaction_df['customer_id'].astype(int)
display(transaction_df)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,25.02.2017,False,Approved,Solex,Standard,medium,medium,71.49,53.62
1,2,3,3120,21.05.2017,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92
2,3,37,402,16.10.2017,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82
3,4,88,3135,31.08.2017,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10
4,5,78,787,01.10.2017,True,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48
...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,51,1018,24.06.2017,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.40
19996,19997,41,127,09.11.2017,True,Approved,Solex,Road,medium,medium,416.98,312.74
19997,19998,87,2284,14.04.2017,True,Approved,OHM Cycles,Standard,medium,medium,1636.90,44.71
19998,19999,6,2764,03.07.2017,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73


In [6]:
# Функция для проверки таблиц
def check_tables(limit=10):
    # Получаем список всех таблиц в схеме public
    cursor.execute("""
        SELECT table_name FROM information_schema.tables 
        WHERE table_schema='public' AND table_type='BASE TABLE'
    """)
    tables = cursor.fetchall()

    print("Список таблиц в базе данных:")
    
    for table in tables:
        table_name = table[0]
        print(f"\n Таблица: {table_name}")

        # Запрашиваем данные из текущей таблицы
        try:
            cursor.execute(f"SELECT * FROM {table_name} LIMIT {limit}")
            rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]  
            
            df = pd.DataFrame(rows, columns=colnames)
            print(df)

        except Exception as e:
            print(f"Ошибка при получении данных из {table_name}: {e}")

In [7]:
conn = pg.connect(dbname='hw2_database', host='localhost', user='postgres', password='12345', port=5432)
cursor = conn.cursor()

In [8]:
check_tables()

Список таблиц в базе данных:


In [9]:
create_tables_query = '''

CREATE TABLE Customer (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    gender VARCHAR(30),
    dob VARCHAR(50),
    job_title VARCHAR(50),
    job_industry_category VARCHAR(50),
    wealth_segment VARCHAR(50),
    deceased_indicator VARCHAR(30),
    owns_car VARCHAR(30),
    address VARCHAR(50),
    postcode VARCHAR(30),
    state VARCHAR(30),
    country VARCHAR(30),
    property_valuation INT
);

CREATE TABLE Transaction (
    transaction_id INT PRIMARY KEY,
    product_id INT,
    customer_id INT,
    transaction_date VARCHAR(30),
    online_order VARCHAR(30),
    order_status VARCHAR(30),
    brand VARCHAR(30),
    product_line VARCHAR(30),
    product_class VARCHAR(30),
    product_size VARCHAR(30),
    list_price FLOAT,
    standard_cost FLOAT,
    FOREIGN KEY (customer_id) REFERENCES Customer(customer_id) ON DELETE CASCADE
)
'''

In [10]:
# Выполняем SQL-запрос для создания таблиц
cursor.execute(create_tables_query)
conn.commit()

In [11]:
# Загружаем таблицу Customer
for _, row in customer_df.iterrows():
    cursor.execute("""
        INSERT INTO Customer (
            customer_id, first_name, last_name, gender, dob, job_title, 
            job_industry_category, wealth_segment, deceased_indicator, owns_car, 
            address, postcode, state, country, property_valuation
        ) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (customer_id) DO NOTHING;
    """, (
        row['customer_id'], row['first_name'], row['last_name'], row['gender'], row['DOB'],
        row['job_title'], row['job_industry_category'], row['wealth_segment'],
        row['deceased_indicator'], row['owns_car'], row['address'], row['postcode'],
        row['state'], row['country'], row['property_valuation']
    ))

conn.commit()



In [12]:
# Загружаем таблицу Transaction
for _, row in transaction_df.iterrows():
    cursor.execute("""
        INSERT INTO Transaction (
            transaction_id, product_id, customer_id, transaction_date, online_order, 
            order_status, brand, product_line, product_class, product_size, 
            list_price, standard_cost
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (transaction_id) DO NOTHING;
    """, (
        row['transaction_id'], row['product_id'], row['customer_id'], row['transaction_date'],
        row['online_order'], row['order_status'], row['brand'], row['product_line'],
        row['product_class'], row['product_size'], row['list_price'], row['standard_cost']
    ))
conn.commit()

In [13]:
check_tables()

Список таблиц в базе данных:

 Таблица: customer
   customer_id      first_name  last_name  gender         dob  \
0            1         Laraine  Medendorp       F  1953-10-12   
1            2             Eli    Bockman    Male  1980-12-16   
2            3           Arlin     Dearle    Male  1954-01-20   
3            4          Talbot        NaN    Male  1961-10-03   
4            5  Sheila-kathryn     Calton  Female  1977-05-13   
5            6            Curr  Duckhouse    Male  1966-09-16   
6            7            Fina     Merali  Female  1976-02-23   
7            8             Rod      Inder    Male  1962-03-30   
8            9            Mala       Lind  Female  1973-03-10   
9           10        Fiorenze    Birdall  Female  1988-10-11   

                              job_title job_industry_category  \
0                   Executive Secretary                Health   
1                Administrative Officer    Financial Services   
2                    Recruiting Manager 

## Оцениваемые задания

### Вывести все уникальные бренды, у которых стандартная стоимость выше 1500 долларов. (1 балл)

In [14]:
cursor.execute("""
        SELECT DISTINCT brand
        FROM Transaction
        WHERE standard_cost > 1500;
    """)

results = cursor.fetchall()
for row in results:
    print(row)

('NaN',)
('Solex',)
('Giant Bicycles',)
('OHM Cycles',)
('Trek Bicycles',)


### Вывести все подтвержденные транзакции за период '2017-04-01' по '2017-04-09' включительно. (1 балл)

In [15]:
cursor.execute("""
    SELECT *
    FROM Transaction
    WHERE order_status = 'Approved'
      AND TO_DATE(transaction_date, 'DD.MM.YYYY') BETWEEN '2017-04-01' AND '2017-04-09';
""")
results = cursor.fetchall()
for row in results:
    print(row)



(17, 79, 2426, '03.04.2017', 'false', 'Approved', 'Norco Bicycles', 'Standard', 'medium', 'medium', 1555.58, 818.01)
(19, 54, 2268, '06.04.2017', 'true', 'Approved', 'WeareA2B', 'Standard', 'medium', 'medium', 1292.84, 13.44)
(23, 37, 2001, '08.04.2017', 'true', 'Approved', 'OHM Cycles', 'Standard', 'low', 'medium', 1793.43, 248.82)
(83, 0, 3398, '01.04.2017', 'true', 'Approved', 'OHM Cycles', 'Standard', 'medium', 'medium', 235.63, 125.07)
(89, 0, 2682, '04.04.2017', 'true', 'Approved', 'OHM Cycles', 'Road', 'high', 'large', 12.01, 7.21)
(126, 53, 773, '01.04.2017', 'false', 'Approved', 'OHM Cycles', 'Standard', 'medium', 'medium', 795.34, 101.58)
(146, 41, 3261, '06.04.2017', 'true', 'Approved', 'Solex', 'Road', 'medium', 'medium', 416.98, 312.74)
(154, 3, 3383, '06.04.2017', 'true', 'Approved', 'Trek Bicycles', 'Standard', 'medium', 'large', 2091.47, 388.92)
(220, 10, 1502, '09.04.2017', 'true', 'Approved', 'WeareA2B', 'Touring', 'medium', 'medium', 1466.68, 363.25)
(228, 13, 821, '

### Вывести все профессии у клиентов из сферы IT или Financial Services, которые начинаются с фразы 'Senior'.

In [16]:
cursor.execute("""
        SELECT DISTINCT job_title
        FROM Customer
        WHERE job_industry_category IN ('IT', 'Financial Services')
          AND job_title LIKE 'Senior%';
    """)
results = cursor.fetchall()
for row in results:
    print(row)


('Senior Cost Accountant',)
('Senior Developer',)
('Senior Editor',)
('Senior Financial Analyst',)
('Senior Quality Engineer',)
('Senior Sales Associate',)


### Вывести все бренды, которые закупают клиенты, работающие в сфере Financial Services (1 балл)

In [17]:
cursor.execute("""
        SELECT DISTINCT t.brand
        FROM Transaction t
        JOIN Customer c ON t.customer_id = c.customer_id
        WHERE c.job_industry_category = 'Financial Services';
    """)
results = cursor.fetchall()
for row in results:
    print(row)

('Giant Bicycles',)
('NaN',)
('Norco Bicycles',)
('OHM Cycles',)
('Solex',)
('Trek Bicycles',)
('WeareA2B',)


### Вывести 10 клиентов, которые оформили онлайн-заказ продукции из брендов 'Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles'. (1 балл)

In [18]:
cursor.execute("""
        SELECT DISTINCT c.*
        FROM Customer c
        JOIN Transaction t ON c.customer_id = t.customer_id
        WHERE t.online_order = 'true'
          AND t.brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles')
        LIMIT 10;
    """
)
results = cursor.fetchall()
for row in results:
    print(row)

(1, 'Laraine', 'Medendorp', 'F', '1953-10-12', 'Executive Secretary', 'Health', 'Mass Customer', 'N', 'Yes', '060 Morning Avenue', '2016', 'New South Wales', 'Australia', 10)
(2, 'Eli', 'Bockman', 'Male', '1980-12-16', 'Administrative Officer', 'Financial Services', 'Mass Customer', 'N', 'Yes', '6 Meadow Vale Court', '2153', 'New South Wales', 'Australia', 10)
(3, 'Arlin', 'Dearle', 'Male', '1954-01-20', 'Recruiting Manager', 'Property', 'Mass Customer', 'N', 'Yes', '0 Holy Cross Court', '4211', 'QLD', 'Australia', 9)
(5, 'Sheila-kathryn', 'Calton', 'Female', '1977-05-13', 'Senior Editor', 'NaN', 'Affluent Customer', 'N', 'Yes', '9 Oakridge Court', '3216', 'VIC', 'Australia', 9)
(6, 'Curr', 'Duckhouse', 'Male', '1966-09-16', 'NaN', 'Retail', 'High Net Worth', 'N', 'Yes', '4 Delaware Trail', '2210', 'New South Wales', 'Australia', 9)
(7, 'Fina', 'Merali', 'Female', '1976-02-23', 'NaN', 'Financial Services', 'Affluent Customer', 'N', 'Yes', '49 Londonderry Lane', '2650', 'New South Wales

### Вывести всех клиентов, у которых нет транзакций. (1 балл)

In [19]:
cursor.execute("""
        SELECT *
        FROM Customer
        WHERE customer_id NOT IN (SELECT DISTINCT customer_id FROM Transaction);
    """
)
results = cursor.fetchall()
for row in results:
    print(row)

(852, 'Andie', 'Bonney', 'Female', '2000-11-04', 'Compensation Analyst', 'Financial Services', 'Affluent Customer', 'N', 'Yes', '94 Anhalt Way', '3139', 'VIC', 'Australia', 7)
(869, 'Addia', 'Abels', 'Female', '1984-03-11', 'Account Representative I', 'Financial Services', 'High Net Worth', 'N', 'Yes', '02377 Maywood Trail', '2287', 'NSW', 'Australia', 7)
(1373, 'Shaylynn', 'Epsley', 'Female', '1958-09-23', 'Director of Sales', 'Financial Services', 'Mass Customer', 'N', 'Yes', '0 Grasskamp Pass', '3170', 'VIC', 'Australia', 10)
(2074, 'Roslyn', 'Rawdall', 'Female', '1997-06-11', 'NaN', 'Financial Services', 'Mass Customer', 'N', 'No', '95483 Washington Junction', '2505', 'NSW', 'Australia', 9)
(2660, 'Hunt', 'Scollard', 'Male', '1963-11-15', 'NaN', 'Retail', 'High Net Worth', 'N', 'Yes', '359 Briar Crest Road', '2155', 'NSW', 'Australia', 10)
(3229, 'Carrol', 'Gheorghie', 'Male', '1964-10-18', 'Sales Representative', 'Retail', 'High Net Worth', 'N', 'Yes', '44 Service Alley', '3023', 

### Вывести всех клиентов из IT, у которых транзакции с максимальной стандартной стоимостью. (1 балл)

In [20]:
cursor.execute("""
        WITH MaxCost AS (
            SELECT MAX(standard_cost) AS max_cost FROM Transaction
        )
        SELECT DISTINCT c.*
        FROM Customer c
        JOIN Transaction t ON c.customer_id = t.customer_id
        JOIN MaxCost m ON t.standard_cost = m.max_cost
        WHERE c.job_industry_category = 'IT';
    """
)
results = cursor.fetchall()
for row in results:
    print(row)

(34, 'Jephthah', 'Bachmann', 'U', '1843-12-21', 'Legal Assistant', 'IT', 'Affluent Customer', 'N', 'No', '90 Lawn Parkway', '4805', 'QLD', 'Australia', 4)
(168, 'Reggie', 'Broggetti', 'U', 'NaN', 'General Manager', 'IT', 'Affluent Customer', 'N', 'Yes', '35386 Mockingbird Road', '3749', 'VIC', 'Australia', 4)
(255, 'Keeley', 'Kruger', 'Female', '1954-01-15', 'Desktop Support Technician', 'IT', 'Mass Customer', 'N', 'No', '8208 Parkside Point', '2320', 'NSW', 'Australia', 6)
(448, 'Robin', 'Crack', 'Female', '1995-04-26', 'Software Engineer III', 'IT', 'High Net Worth', 'N', 'No', '24919 Artisan Court', '3015', 'Victoria', 'Australia', 8)
(968, 'Wayne', 'Woodfin', 'Male', '1955-06-16', 'Accountant IV', 'IT', 'High Net Worth', 'N', 'Yes', '004 Lawn Trail', '4227', 'QLD', 'Australia', 5)
(975, 'Goldarina', 'Rzehorz', 'U', 'NaN', 'Automation Specialist IV', 'IT', 'Mass Customer', 'N', 'No', '938 Monica Park', '3173', 'VIC', 'Australia', 9)
(1182, 'Carolann', 'Ottiwill', 'Female', '1975-01-

### Вывести всех клиентов из сферы IT и Health, у которых есть подтвержденные транзакции за период '2017-07-07' по '2017-07-17'. (1 балл)

In [21]:
cursor.execute("""
        SELECT DISTINCT c.*
        FROM Customer c
        JOIN Transaction t ON c.customer_id = t.customer_id
        WHERE c.job_industry_category IN ('IT', 'Health')
          AND t.order_status = 'Approved'
          AND TO_DATE(t.transaction_date, 'DD.MM.YYYY') BETWEEN '2017-07-07' AND '2017-07-17';
    """
)
results = cursor.fetchall()
for row in results:
    print(row)

(22, 'Deeanne', 'Durtnell', 'Female', '1962-12-10', 'NaN', 'IT', 'Mass Customer', 'N', 'No', '52 Carey Alley', '4740', 'QLD', 'Australia', 5)
(28, 'Fee', 'Zellmer', 'Male', '1973-09-30', 'Senior Quality Engineer', 'Health', 'Affluent Customer', 'N', 'Yes', '2951 Petterle Place', '2756', 'New South Wales', 'Australia', 9)
(41, 'Basilius', 'Coupe', 'Male', '1976-04-14', 'Food Chemist', 'Health', 'Mass Customer', 'N', 'No', '2028 Lakewood Place', '2480', 'New South Wales', 'Australia', 3)
(47, 'Matthew', 'Jeaycock', 'Male', '1992-10-22', 'Registered Nurse', 'Health', 'Affluent Customer', 'N', 'No', '4853 Gulseth Avenue', '4503', 'QLD', 'Australia', 5)
(104, 'Odille', 'Panketh', 'Female', '1978-06-06', 'Automation Specialist II', 'Health', 'Mass Customer', 'N', 'Yes', '0410 Division Junction', '2750', 'New South Wales', 'Australia', 8)
(235, 'Leona', 'Phateplace', 'Female', '1976-12-26', 'Social Worker', 'Health', 'Mass Customer', 'N', 'No', '95 Eliot Street', '4514', 'QLD', 'Australia', 3